## Welcome to the Training_Model notebook

Goals of this notebook:
* Train a Model to classify suspicious firms as such, given their data

###### The Auditor Office of India, officially known as the Comptroller and Auditor General of India (CAG), is a Central Government Agency established by the Indian Constitution with the express purpose of ensuring public funds are being used properly (making sure the gov't isn't being scammed). It does this by auditing Central and State Government accounts.

## Import necessary libraries

In [125]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np

from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression


## Load the Dataset

The dataset is the 'Audit Data' dataset from the UCI Machine Learning Repository

In [126]:
ad = pd.read_csv('../audit_data/audit_risk.csv') # 'ad' for 'audit data'
td = pd.read_csv('../audit_data/trial.csv') # 'td' for 'trial data'

In [127]:
print(ad.columns) # to see the different columns that ad and td have
print(td.columns)

ad.sample(10)


Index(['Sector_score', 'LOCATION_ID', 'PARA_A', 'Score_A', 'Risk_A', 'PARA_B',
       'Score_B', 'Risk_B', 'TOTAL', 'numbers', 'Score_B.1', 'Risk_C',
       'Money_Value', 'Score_MV', 'Risk_D', 'District_Loss', 'PROB', 'RiSk_E',
       'History', 'Prob', 'Risk_F', 'Score', 'Inherent_Risk', 'CONTROL_RISK',
       'Detection_Risk', 'Audit_Risk', 'Risk'],
      dtype='object')
Index(['Sector_score', 'LOCATION_ID', 'PARA_A', 'SCORE_A', 'PARA_B', 'SCORE_B',
       'TOTAL', 'numbers', 'Marks', 'Money_Value', 'MONEY_Marks', 'District',
       'Loss', 'LOSS_SCORE', 'History', 'History_score', 'Score', 'Risk'],
      dtype='object')


,Sector_score,LOCATION_ID,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
303,2.37,28,1.29,0.4,0.516,0.25,0.2,0.050,1.54,5.0,...,0.4,0,0.2,0.0,2.2,1.966,0.4,0.5,0.3932,0
477,1.85,19,0.00,0.2,0.000,0.00,0.2,0.000,0.00,5.0,...,1.2,0,0.2,0.0,2.4,2.200,1.2,0.5,1.3200,1
512,21.61,8,0.34,0.2,0.068,0.36,0.2,0.072,0.70,5.0,...,0.4,0,0.2,0.0,2.0,1.546,0.4,0.5,0.3092,0
162,3.41,13,5.61,0.6,3.366,1.56,0.2,0.312,7.17,5.0,...,0.4,0,0.2,0.0,2.4,5.884,0.4,0.5,1.1768,1
546,59.85,11,0.00,0.2,0.000,0.00,0.2,0.000,0.00,5.0,...,0.4,0,0.2,0.0,2.0,1.540,0.4,0.5,0.3080,0
324,2.37,2,2.18,0.6,1.308,0.93,0.2,0.186,3.11,5.0,...,1.2,0,0.2,0.0,2.8,3.694,1.2,0.5,2.2164,1
114,3.41,34,4.25,0.6,2.550,45.51,0.6,27.306,49.76,5.5,...,0.4,0,0.2,0.0,4.2,35.736,0.4,0.5,7.1472,1
49,3.89,22,2.95,0.6,1.770,6.56,0.4,2.624,9.51,5.0,...,0.4,0,0.2,0.0,3.4,99.946,0.4,0.5,19.9892,1
140,3.41,16,12.68,0.6,7.608,41.00,0.6,24.600,53.68,5.5,...,0.4,0,0.2,0.0,4.4,41.936,0.4,0.5,8.3872,1
36,3.89,20,9.01,0.6,5.406,19.82,0.6,11.892,28.83,5.0,...,0.4,0,0.2,0.0,3.8,22.566,0.4,0.5,4.5132,1


## Use PCA to determine the relative importance of each feature

Just for funsies, not sure it will work. Will do for both ad and td

In [128]:
from IPython.display import display #display module for jupyter notebook, allowing me to manually force Jupyter to display in the nice UI form

#INVESTGATING 'ad' FOR NaN values / READYING IT FOR PCA
print('Investigating \'ad\' for NaN values and readying it for pca:\n')

ad.loc[ad.LOCATION_ID.isin(['LOHARU', 'NUH', 'SAFIDON'])] #find the row index numbers where these strings are in LOCATION_ID

#print(ad.head(10))
ad_stringless = ad.copy().drop([351,355,367], axis=0)
print(f'Shape of the original ad is {ad.shape}')
print(f'Shape after removing the 3 rows with Loharu Nuh and Safidon is {ad_stringless.shape}')
pd.set_option('display.max_columns', None)
#print(ad_stringless.isna().any(axis=0)) # checks if there's an NA in any column, and if so, labels that column as "True"
display(ad_stringless[ad_stringless.isna().any(axis=1)]) # returns the rows with at least 1 box that is NA, in the nice UI form

pd.reset_option('display.max_columns')

ad_stringless = ad_stringless.dropna()
print(f'Shape after dropping na is {ad_stringless.shape}')


# PCA SECTION - USING PCA TO FIND HOW IMPORTANT EACH COLUMN IS
print('Pca section - using pca to find the relative importance(explained variance) of each column:\n')

# PCA and Pandas are already imported

pca = PCA()

pca.fit(ad_stringless)
ad_explained_variance_ratio = pca.explained_variance_ratio_ # must be ratio, otherwise it's just absolute values that don't really mean much to us

print('Audit Data (ad) columns and explained variance:\n')
print(ad.columns + '\n')
np.set_printoptions(precision=4, suppress=True) 
print(ad_explained_variance_ratio)


Investigating 'ad' for NaN values and readying it for pca:

Shape of the original ad is (776, 27)
Shape after removing the 3 rows with Loharu Nuh and Safidon is (773, 27)


,Sector_score,LOCATION_ID,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,Risk_C,Money_Value,Score_MV,Risk_D,District_Loss,PROB,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
642,55.57,4,0.23,0.2,0.046,0.0,0.2,0.0,0.23,5.0,0.2,1.0,NaN,0.2,0.0,2,0.2,0.4,0,0.2,0.0,2.0,1.446,0.4,0.5,0.2892,0


Shape after dropping na is (772, 27)
Pca section - using pca to find the relative importance(explained variance) of each column:

Audit Data (ad) columns and explained variance:

Index(['Sector_score\n', 'LOCATION_ID\n', 'PARA_A\n', 'Score_A\n', 'Risk_A\n',
       'PARA_B\n', 'Score_B\n', 'Risk_B\n', 'TOTAL\n', 'numbers\n',
       'Score_B.1\n', 'Risk_C\n', 'Money_Value\n', 'Score_MV\n', 'Risk_D\n',
       'District_Loss\n', 'PROB\n', 'RiSk_E\n', 'History\n', 'Prob\n',
       'Risk_F\n', 'Score\n', 'Inherent_Risk\n', 'CONTROL_RISK\n',
       'Detection_Risk\n', 'Audit_Risk\n', 'Risk\n'],
      dtype='object')
[0.6288 0.3171 0.0336 0.0123 0.0052 0.0027 0.0001 0.0001 0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.    ]


In [ ]:
# td version of the section above

## Discoveries made about ad and td

1. In both 'ad' and 'td', there are 3 rows where the LOCATION_ID is string, so it is not a float, and hence not compatible with pca. Hence, they were removed.
2. In 'ad', for some reason, in row 642, 'Money_Value' is NaN. So, I removed the row.
3. For 'ad', it appears that the location and first couple of 'para's hold the majority of the file's data variety (explained variance)
    This means that the location and first 2 'para's vary the most throughout the data. The location makes sense, but this could also imply that  columns 9-27 are relatively similar across firms. We can't entirely be sure without more information or investigation.

Note: I am removing pca-incompatible rows because I believe that in the face of 770+ rows, those individual rows probably don't matter too much. In other words, removing them is worth it for the ability to use PCA to learn more about the columns in the original data.

Note to self: in the next section, do logistic regression on ad and td without pca.

then, in the section after that, do logistic regression on ad and td post-pca. the pca work should have been completed above, so just grab ad_stringless and td_stringless to throw into train_test_split. Have to isolate the 'Risk' row though(pull out of these dataframes and separate into its own series), since it is the target.